<img src="https://datasciencedegree.wisconsin.edu/wp-content/themes/data-gulp/images/logo.svg" width="300">


# Assignment 11

This assignment is probably the most involved yet.  You will process a large file through Python, and then pass it to R for statistical analysis.

We will analyze Amazon reviews to determine what characteristics make them most helpful.

## Problem 1(a).  Reading Amazon Reviews.

🎯 Download the file of Amazon gourmet food reviews from the [Stanford Large Network Dataset Collection](https://snap.stanford.edu/data/web-FineFoods.html).   (Your computer may already have a utility installed that can unzip the archive as a text file; if not, [7-zip](http://www.7-zip.org/) is a useful utility for Windows. You can also use an online utility by doing a web search for: ``open .gz files online``.)

🎯 Create a pandas DataFrame object with the following entries for each review:

1. Product ID
2. Number of people who voted this review helpful
3. Total number of people who rated this review
4. Reviewer's score rating of the product
5. Text of the review -- this will be dropped before you write your data file and port to R.

For the second and third of these, the information will be given in the file as ```1/5```, which would correspond to 1 vote for helpful out of 5 people who rated the review.

---

###### Examples

Consider this review:

    product/productId: B001E4KFG0
    review/userId: A3SGXH7AUHU8GW
    review/profileName: delmartian
    review/helpfulness: 1/1
    review/score: 5.0
    review/time: 1303862400
    review/summary: Good Quality Dog Food
    review/text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

The fraction for `review/helpfulness` for delmartian's review here is 1/1.  One person voted on whether it was a helpful review, and that one person found it helpful.

---

    product/productId: B001GVISJM
    review/userId: A2MUGFV2TDQ47K
    review/profileName: Lynrie "Oh HELL no"
    review/helpfulness: 4/5
    review/score: 5.0
    review/time: 1268352000
    review/summary: Strawberry Twizzlers - Yummy
    review/text: The Strawberry Twizzlers are my guilty pleasure - yummy. Six pounds will be around for a while with my son and I.

This review was rated for helpfulness by a total of 5 people, but only four of them found it helpful -- the other person didn't think it was a useful review.

---

    product/productId: B001GVISJM
    review/userId: A3IV7CL2C13K2U
    review/profileName: Greg
    review/helpfulness: 0/0
    review/score: 5.0
    review/time: 1318032000
    review/summary: Home delivered twizlers
    review/text: Candy was delivered very fast and was purchased at a reasonable price.  I was home bound and unable to get to a store so this was perfect for me.
    
Zero people voted on whether Greg's review was helpful.  Of these zero people, zero found it helpful. 

---

###### Notes

I strongly suggest that you take some smaller portion of the file off the top for testing and development.  On a Mac, in the [terminal](https://www.macworld.co.uk/feature/mac-software/how-use-terminal-on-mac-3608274/), change directories to the location of this repo, and run

    head -n 500 finefoods.csv > finefoods_test.csv
    
But, change that 500 to some number such that you don't have a broken review at the bottom!  Windows students, use the [Powershell](https://en.wikipedia.org/wiki/PowerShell). (regular old `cmd` is terrible)

---

Here are the first 10 reviews, correctly parsed:

![first_10](first_10.jpg)

In [1]:
import pandas as pd 
import numpy as np

fileLoc = "C:/Users/benja/OneDrive/Documents/GitHub/ds710fall2018assignment11/foods.txt"

with open (fileLoc) as food_file:

    dict_list = []
    column_names = ["Num_helpful", "Num_voters","Review_text","Product_ID","Review_score"]
    prodID = "" 
    numvot = ""
    numhelp = ""
    revscore = ""
    revtxt = ""
    
    
    for line in food_file:
        
        nextLine = food_file.readline()
        
        
        numhelpStart = nextLine.find('review/helpfulness:')
        prodStart = nextLine.find('product/productId:')
        reviewStart = nextLine.find('review/text:')
        scoreStart = nextLine.find('review/score:')
        
        if numhelpStart != -1:
            if numhelp:
                dict_list.append(dict(zip(column_names, [numhelp, numvot, revtxt, prodID, revscore])))
            forboth = nextLine.strip("review/helpfulness: ")
            sep = '/'
            numhelp = forboth.split(sep, 1)[0]
            numhelp = numhelp.strip()
            numvot = forboth.split(sep, 1)[1]
            numvot = numvot.strip()
        elif reviewStart != -1:
            revtxt = nextLine.strip("review/text: ")
            revtxt = revtxt.rstrip()
        elif prodStart != -1:
            prodID = nextLine.strip("product/productId: ")
            prodID = prodID.strip()
        elif scoreStart != -1:
            revscore = nextLine.strip("review/score: ")
            revscore = revscore.rstrip()
            revscore = eval(revscore)

food_df = pd.DataFrame(dict_list)             

## Problem 1(b).  Analyzing review text.

🎯 Add columns to your DataFrame for 
* the length of a review, 
* the number of exclamation points in a review, and 
* the fraction of people who rated a review helpful. 

###### Notes 

You should calculate the fraction who rated a review helpful using two of the columns you made in part 1a.  
* You must convert from text fractions to floating point numbers.  For example, a ratio of 1 helpful rating out of 5 total ratings should be entered as 0.2, not the string ```1/5```.  This floating point number must not be rounded.
* If no people voted on whether a problem was helpful, the corresponding entry in your percentage helpful column should be ```NaN```.  0/0 should generally turn into a `nan` when computing.

In [2]:
food_df['Review_length'] = food_df["Review_text"].apply(len)

food_df['num_exclamations'] = food_df["Review_text"].apply(lambda x: x.count("!"))

food_df['fract_help'] = food_df['Num_helpful'].astype(float) / food_df['Num_voters'].astype(float)

## Problem 1(c).  Summary statistics.

🎯 Compute these using Python:

1. How many reviews are in the data set?  
2. What is the average length of a review (in characters)?  
3. What is the average rating?  
4. What is the greatest number of exclamation marks used in a single review?  

Use the pandas package to answer these questions.

In [3]:
food_df.describe(include=['object'])
#There are 217227 unique reviews in the data frame

food_df['Review_length'].mean()
#The average character length of a review is 435.72 characters

food_df['Review_score'].mean()
#The average rating is 4.18

food_df['num_exclamations'].max()
#The greatest number of exclamation marks used in a review is 71        


71

## Problem 1(d).  Export.

🎯 Save your DataFrame as a `.csv` file suitable for future analysis in R.  

###### Requirements

* Your .csv file must not include the review text column, as the presence of commas and quotation marks will make reading the file difficult.  
* You should also convert entries from ```NaN``` to the empty string before saving.

In [4]:
food_df = food_df.replace(np.nan, '', regex=True)
food_df.to_csv('food_pandas.csv', columns=["Num_helpful", "Num_voters", "Product_ID", "Review_score", "Review_length", "num_exclamations", "fract_help"],  index = False)